In [1]:
import cv2
import mediapipe as mp
import numpy as np
import time
from sklearn.metrics.pairwise import cosine_similarity

target_vector_1 = None

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

In [25]:
def get_normalized_landmarks (landmarks):
                if landmarks is None:
                     return None
                # Convert landmarks to a numpy array (21 points, 3 coords)
                # landmarks format: [[x1, y1, z1], [x2, y2, z2], ...]
                points = np.array([[lm.x, lm.y, lm.z] for lm in landmarks.landmark[11:23]])
                
                # 1. Translation Invariance: Make Wrist (index 0) the origin
                left_shoulder = points[0]
                right_shoulder = points[1]
                midpoint = (left_shoulder+right_shoulder)/2
                relative_points = points - midpoint
                
                # 2. Scale Invariance: Normalize by the size of the hand
                # We use the max distance from wrist to any point as the scale factor
                max_dist = np.max(np.linalg.norm(relative_points, axis=1))
                if max_dist == 0:
                    max_dist = 1
                normalized_points = relative_points / max_dist
                
                # Flatten into a 1D vector (size 63) for cosine similarity
                return normalized_points.flatten()

In [33]:
global target_vector_1
cap=cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_BUFFERSIZE, 1)
with mp_holistic.Holistic(min_detection_confidence=.7, min_tracking_confidence=0.7) as holistic:
        while cap.isOpened():
            ret, Frame = cap.read()
            
            image = cv2.cvtColor(Frame, cv2.COLOR_BGR2RGB)
            results = holistic.process(image)
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            #pose
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,mp_drawing.DrawingSpec((230,216,173),2,2))
            

            # --- Usage ---

            # 1. Record your "Target" gesture (e.g., a Peace Sign) once and save it
            if results.pose_landmarks is None:
                if cv2.waitKey(10) & 0xFF == ord('d'):
                    break
            else : 
                target_vector_1 = get_normalized_landmarks(results.pose_landmarks)
                if cv2.waitKey(10) & 0xFF == ord('d'):
                    time.sleep(2)
                    break


            # 2. In your loop:
            #current_vector = get_normalized_landmarks(current_frame_landmarks)

            # 3. Reshape for sklearn (expects 2D array)
            #similarity = cosine_similarity([target_vector], [current_vector])

            #if similarity[0][0] > 0.95: # Threshold
            #    print("Gesture Detected!")
            cv2.imshow("Raw_Webcam_Feed", image)

            if cv2.waitKey(10) & 0xFF == ord('d'):
                break

cap.release()
cv2.destroyAllWindows()

In [ ]:
global target_vector_1
cap=cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_BUFFERSIZE, 1)
with mp_holistic.Holistic(min_detection_confidence=.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, Frame = cap.read()
        if not ret:
            break

        image = cv2.cvtColor(Frame, cv2.COLOR_BGR2RGB)
        results = holistic.process(image)
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, mp_drawing.DrawingSpec((230,216,173),2,2))
        cv2.imshow("Raw_Webcam_Feed", image)
        

        if results.pose_landmarks is None:
            if cv2.waitKey(10) & 0xFF == ord('d'):
                break
            continue
        
        current_vector_1 = get_normalized_landmarks(results.pose_landmarks)    
        sim_1 = cosine_similarity([target_vector_1], [current_vector_1])[0][0]
        
        if (sim_1 > 0.98):
            cv2.destroyAllWindows()
            cap = cv2.VideoCapture("assets/nub67.gif")
            loop_count = 0
            max = 3
            while True:
                ret, Frame = cap.read()
                if not ret:
                    loop_count+=1
                    if loop_count<max:
                        cap.set(cv2.CAP_PROP_POS_FRAMES,0)
                        continue
                    else :
                        print("you got six-sevennnnnnned")
                        break

                cv2.imshow("Gif", Frame)

                if cv2.waitKey(200) & 0xFF == ord('d'):
                    break

            cap.release()
            cv2.destroyAllWindows()
        
        if cv2.waitKey(1) & 0xFF == ord('d'):
            break    

cap.release()
cv2.destroyAllWindows()

you got six-sevennnnnnned


In [32]:
cap.release()
cv2.destroyAllWindows()